In [1]:
from ipyleaflet import Map, TileLayer, LayersControl, basemaps, basemap_to_tiles, WidgetControl, LegendControl, GeoJSON
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
import IPython
import requests
import geojson
import json

In [2]:
# Base host url InVEST is expected to be serving
host_url = 'http://localhost:8081'
# Load information dynamically created by most recent schisto model run.
nb_json_config_path = f'{host_url}/output/nb-json-config.json'

# Get the layers from the most recent model run to display
nb_json_config = requests.get(nb_json_config_path).json()
layer_name_list = []
for layer in nb_json_config['layers']:
    layer_name_list.append(layer + "_tiles")

# Get the plot png paths from the most recent model run
plot_root_path = f'{host_url}/intermediate/plot_previews/'
plot_png_list = [plot_root_path + plot_name for plot_name in nb_json_config['plot_paths']]

# Get map center
map_center = nb_json_config['aoi_center']
# Get aoi geojson
aoi_geojson_path = f'{host_url}/output/{nb_json_config['aoi_geojson']}'

In [3]:
# Get legend colors
# Hard coded for now, could read in from generated color-profiles
hab_suit_colorbar = [(0, "#ffffb2"), (0.25, "#fecc5c"), (0.5, "#fd8d3c"), (0.75, "#f03b20"), (1.0, "#bd0026")]
pop_suit_colorbar = [(0, "#f7fbff"), (0.20, "#d1e2f3"), (0.40, "#9ac8e0"), (0.60, "#529dcc"), (0.80, "#1d6cb1"), (1.0, "#08306b")]
empty_colorbar = [(0, "#a4a6a5"), (1.0, "#a4a6a5")]

# LEGEND CONTROL
# This method is taken from https://github.com/jupyter-widgets/ipyleaflet/issues/706
import matplotlib.pyplot as plt; import matplotlib as mpl
import io
colorbar_dict = {}
for key, colorbar in zip(['hab', 'pop', 'empty'], [hab_suit_colorbar, pop_suit_colorbar, empty_colorbar]):
    fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')
    norm = mpl.colors.Normalize(vmin=0.0, vmax=1.0)
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list('mymap', colorbar)
    fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=my_cmap), cax=ax, orientation='horizontal',
        label='Risk', ticks=[x[0] for x in colorbar])
    f = io.BytesIO()
    plt.savefig(f, bbox_inches='tight', format='png')
    image = f.getvalue()
    colorbar_dict[key] = image
    plt.close()


In [4]:
# Create a GridBox from the plot widgets
plot_widgets = []
for plot_image in plot_png_list:
    image = IPython.display.Image(plot_image)
    tmp_widget = widgets.Image(
        value=image.data,
        format='png',
    )
    plot_widgets.append(tmp_widget)

plot_grid = widgets.GridBox(plot_widgets, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)", overflow_x='auto'))

In [5]:
# Set up the leaflet map
chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'
# Need to find a way to programmatically get center for diff locations
m = Map(center=map_center, zoom=8, scroll_wheel_zoom=True, layout=Layout(height='600px'))

In [6]:
# Group layers for easier visualization in map control
water_risk_layers = []
custom_risk_layers = []
population_risk_layers = []
absolute_risk_layers = []
relative_risk_layers = []
# Map layer names to TileLayer representation
checkbox_layers_map = {}
checkbox_legend_map = {}

# Add NDVI layers
for season in ['dry', 'wet']:
    # only display layers from the most recent model run
    base_tile_dir = f'ndvi_suit_{season}_tiles'
    if base_tile_dir in layer_name_list:
        ndvi_tile_url = TileLayer(
            url=f'{host_url}/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
            name=f"ndvi {season}",
            attribution="mine", #TODO: do we need anything here?
            min_zoom=1,
            max_zoom=18,
            min_native_zoom=1,
            max_native_zoom=14,)
        water_risk_layers.append(f'ndvi_suit_{season}')
        checkbox_layers_map[f'ndvi_suit_{season}'] = ndvi_tile_url
        checkbox_legend_map[f'ndvi_suit_{season}'] = 'hab'

# Add water temperature layers
for risk_type in ['parasite', 'snail']:
    for season in ['dry', 'wet']:
        # only display layers from the most recent model run
        base_tile_dir = f'{risk_type}_water_temp_suit_{season}_tiles'
        if base_tile_dir in layer_name_list:
            temp_tile_url = TileLayer(
                url=f'{host_url}/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
                name=f"{risk_type} water temp {season}",
                attribution="mine",
                min_zoom=1,
                max_zoom=18,
                min_native_zoom=1,
                max_native_zoom=14,)
            water_risk_layers.append(f'{risk_type}_water_temp_suit_{season}')
            checkbox_layers_map[f'{risk_type}_water_temp_suit_{season}'] = temp_tile_url
            checkbox_legend_map[f'{risk_type}_water_temp_suit_{season}'] = 'hab'

# Add custom layers
for custom_input in ['one', 'two', 'three']:
    # only display layers from the most recent model run
    base_tile_dir = f'custom_suit_{custom_input}_tiles'
    if base_tile_dir in layer_name_list:
        custom_tile_url = TileLayer(
            url=f'{host_url}/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
            name=f"custom {custom_input}",
            attribution="mine", #TODO: do we need anything here?
            min_zoom=1,
            max_zoom=18,
            min_native_zoom=1,
            max_native_zoom=14,)
        custom_risk_layers.append(f'custom_suit_{custom_input}')
        checkbox_layers_map[f'custom_suit_{custom_input}'] = custom_tile_url
        checkbox_legend_map[f'custom_suit_{custom_input}'] = 'hab'
            
# Add population risk layers
population_risk_tile_url = TileLayer(
    url=f'{host_url}/output/rural_urbanization_suit_tiles/{{z}}/{{x}}/{{y}}.png',
    name="Population risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
population_risk_layers.append('rural_urbanization_suit')
checkbox_layers_map['rural_urbanization_suit'] = population_risk_tile_url
checkbox_legend_map['rural_urbanization_suit'] = 'pop'

# Add weighted mean risk layer
combined_risk_tile_url = TileLayer(
    url=f'{host_url}/output/habitat_suit_weighted_mean_tiles/{{z}}/{{x}}/{{y}}.png',
    name="Combined risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
water_risk_layers.append('habitat_suit_weighted_mean')
checkbox_layers_map['habitat_suit_weighted_mean'] = combined_risk_tile_url
checkbox_legend_map['habitat_suit_weighted_mean'] = 'hab'

# Add distance weighted risk (convolution layer)
convolved_risk_tile_url = TileLayer(
    url=f'{host_url}/output/convolved_hab_risk_tiles/{{z}}/{{x}}/{{y}}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
absolute_risk_layers.append('convolved_hab_risk')
checkbox_layers_map['convolved_hab_risk'] = convolved_risk_tile_url
checkbox_legend_map['convolved_hab_risk'] = 'hab'

# Add normalized distance weighted risk
normalized_risk_tile_url = TileLayer(
    url=f'{host_url}/output/normalized_convolved_risk_tiles/{{z}}/{{x}}/{{y}}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=18,
    min_native_zoom=1,
    max_native_zoom=14,)
relative_risk_layers.append('normalized_convolved_risk')
checkbox_layers_map['normalized_convolved_risk'] = normalized_risk_tile_url
checkbox_legend_map['normalized_convolved_risk'] = 'hab'

# Add relative and absolute risk to population outputs
for risk_type, layer_group in zip(['abs', 'rel'], [absolute_risk_layers, relative_risk_layers]):
    risk_to_pop_tile_url = TileLayer(
        url=f'{host_url}/output/risk_to_pop_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=18,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_{risk_type}'] = risk_to_pop_tile_url
    checkbox_legend_map[f'risk_to_pop_{risk_type}'] = 'pop'
    
    risk_to_pop_count_tile_url = TileLayer(
        url=f'{host_url}/output/risk_to_pop_count_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop count {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=18,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_count_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_count_{risk_type}'] = risk_to_pop_count_tile_url
    checkbox_legend_map[f'risk_to_pop_count_{risk_type}'] = 'pop'

In [7]:
%%capture
# Add AOI layer
aoi_geojson = requests.get(aoi_geojson_path).json()
aoi_key = 'area_of_interest'
geojson_layer = GeoJSON(
    data=aoi_geojson,
    name=aoi_key,
    style={
        'opacity': 0.60, 'dashArray': '9', 'fillOpacity': 0.00, 'weight': 2, 'color': 'black',
    },
)
checkbox_layers_map[aoi_key] = geojson_layer
m.add(geojson_layer)

In [8]:
default_layer_key = "risk_to_pop_count_abs"

# Default colorbar
colorbar_widget = widgets.Image(
    value=colorbar_dict[checkbox_legend_map[default_layer_key]], format='png', 
    layout=Layout(object_fit='contain', margin='0px 0px 0px 0px'))

# Legend widget container
legend_container = widgets.Accordion(
    children=[colorbar_widget], titles=(f"Legend: {default_layer_key}",), 
    layout=Layout(max_width='350px', padding='0px 0px 0px 0px',))

# Track which layers are selected in a stack-like data structure
layer_stack = [default_layer_key]
# out is useful for debugging. We can capture the event outputs
# with the decorator.
out = widgets.Output()

@out.capture()
def layer_visible_switch(event):
    widget = event['owner']
    desc_id = widget.description
    print(desc_id)

    # Treat AOI as a basemap and don't include it in layer_stack
    if desc_id == aoi_key:
        layer_tile_url = checkbox_layers_map[desc_id]
        layer_tile_url.visible = event['new']    
            
    else:
        if event['new']:
            layer_stack.append(desc_id)
        else:
            layer_stack.pop(layer_stack.index(desc_id))
    
        layer_tile_url = checkbox_layers_map[desc_id]
        layer_tile_url.visible = event['new']
        cur_layers = m.layers
        # Preserve basemap in the back
        layer_order = [cur_layers[0]]
        
        for layer in cur_layers[1:]:
            # Skip AOI layer, we always want that in the foreground
            if layer.name == aoi_key:
                continue
            # We want the currently toggled layer, whether visible or not, to be last
            if layer.url != layer_tile_url.url:
                layer_order.append(layer)
        layer_order.append(layer_tile_url)
        # Add AOI last, to be in foreground
        layer_order.append(checkbox_layers_map[aoi_key])
        m.layers = layer_order

        print(layer_stack)
        if layer_stack:
            legend_id = layer_stack[-1]
            colorbar_widget.value = colorbar_dict[checkbox_legend_map[legend_id]]
            legend_container.titles = (f"Legend: {legend_id}",)
        else:
            legend_id = "Select layer"
            colorbar_widget.value = colorbar_dict['empty']
            legend_container.titles = (f"Legend: {legend_id}",)
                
layer_groups = [water_risk_layers, population_risk_layers, absolute_risk_layers, relative_risk_layers]
layer_group_sections = ["Water risk layers", "Population risk layers", "Absolute risk layers", "Relative risk layers"]
if custom_risk_layers:
    layer_groups.append(custom_risk_layers)
    layer_group_sections.append("Custom input layers")

widget_list = []
# Add AOI to accordion widget list
aoi_widget = widgets.Checkbox(
                value=True,
                description=geojson_layer.name,
                disabled=False,
                indent=False,
                layout=Layout(padding='0px 0px 0px 10px', margin='0px 0px 0px 0px', width='auto'),
            )
aoi_widget.observe(layer_visible_switch, names='value')
widget_list.append(aoi_widget)

for group_title, group_list in zip(layer_group_sections, layer_groups):
    group_header_widget = widgets.HTML(description="", value=f'<b>{group_title}<b>')
    checkbox_widget_group = []
    # Only expand the accordion menu w/ the active layer
    accordion_active = False
    for layer_key in group_list:
        
        layer_active = False
        # Default a layer to be active and visible
        if layer_key == default_layer_key:
            layer_active = True
            accordion_active = True
            
        tmp_widget = widgets.Checkbox(
                value=layer_active,
                description=layer_key,
                disabled=False,
                indent=False,
                layout=Layout(padding='0px 0px 0px 10px', margin='0px 0px 0px 0px', width='auto'),
            )
        tmp_widget.observe(layer_visible_switch, names='value')
        checkbox_widget_group.append(tmp_widget)
        checkbox_layers_map[layer_key].visible=layer_active
        m.add(checkbox_layers_map[layer_key])
    
    # Only expand the accordion menu w/ the active layer
    acc_idx = None
    if accordion_active:
        acc_idx = 0
    widget_list.append(widgets.Accordion(
                        children=[widgets.VBox(checkbox_widget_group)],
                        titles=(group_title,), selected_index=acc_idx))
    
vbox_widget = widgets.VBox(widget_list, layout=Layout(height='auto', width='auto', overflow_y='auto', padding='0px 0px 0px 0px'))
# Since we're defaulting a layer on, expand the accordion w/ select_index=0
layers_control_acc = widgets.Accordion(
    children=[vbox_widget], titles=("Layers",), 
    layout=Layout(max_height='350px', padding='0px 0px 0px 0px'),
    selected_index=0)

widget_layer_control = WidgetControl(widget=layers_control_acc, position="bottomleft")
widget_legend_control = WidgetControl(widget=legend_container, position="bottomright")

#map_hbox = widgets.HBox([m]) #, layout=Layout(height="70%"))
widgets.VBox([m, plot_grid])

In [9]:
%%capture
m.add(widget_layer_control)
# Add an inital legend control
m.add(widget_legend_control)

In [10]:
#out

Output()